In [1]:
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt
import numpy as np
from time import sleep
import os
import random
from sklearn.metrics import confusion_matrix, classification_report
import cv2

In [2]:
DATADIR = r".\DL_DATA"
CATEGORIES = ["Book", "Cardboard_box", "Cup"]

In [3]:
def make_train_data():
    for category in CATEGORIES:
        path = os.path.join(DATADIR, category)
        class_num = CATEGORIES.index(category)
        for img in os.listdir(path):
            try:
                img_array = cv2.imread(os.path.join(path, img), cv2.IMREAD_GRAYSCALE)
                train_data.append([img_array, class_num])
            except Exception as e: 
                pass

In [4]:
train_data = []
make_train_data()
random.shuffle(train_data)
print(len(train_data))

1012


In [5]:
X = []
y = []

for features, label in train_data:
    X.append(features)
    y.append(label)

X = np.array(X).reshape(-1, 100, 100, 1)
X = X/255.0
y = np.array(y)

In [6]:
class tooGood(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        if(logs.get("accuracy") > 0.9):
            print("\n Desired accuracy reached; training ended.")
            self.model.stop_training = True

In [7]:
callbacks = tooGood()

In [8]:
cnn_model = models.Sequential([
    layers.Conv2D(filters = 64, kernel_size = (3,3), activation = "relu", input_shape = X.shape[1:]),
    layers.MaxPooling2D((2,2)),
    
    layers.Conv2D(filters = 64, kernel_size = (3,3), activation = "relu"),
    layers.MaxPooling2D((2,2)),
    
    layers.Flatten(),
    layers.Dense(640, activation = "relu"),
    layers.Dense(3, activation = "softmax")
    
    #Maybe add another Dense 1 and activation? Maybe activation sigmoid? 
    
])

In [9]:
cnn_model.compile(optimizer = "adam", 
             loss = "sparse_categorical_crossentropy", 
            #Maybe do binary_crossentropy? if it's greyscale... 
             metrics = ["accuracy"])

In [10]:
#Remember to change the val split
cnn_model.fit(X, y, batch_size = 20, validation_split = 0.1, epochs = 10, callbacks = [callbacks])

Epoch 1/10
46/46 [==============================] - 9s 197ms/step - loss: 0.4337 - accuracy: 0.8637 - val_loss: 0.0374 - val_accuracy: 0.9902
Epoch 2/10
46/46 [==============================] - ETA: 0s - loss: 0.0247 - accuracy: 0.9945
 Desired accuracy reached; training ended.
46/46 [==============================] - 9s 186ms/step - loss: 0.0247 - accuracy: 0.9945 - val_loss: 0.0398 - val_accuracy: 0.9902


In [11]:
cnn_model.save(r".\model_data")

INFO:tensorflow:Assets written to: .\model_data\assets
